In [23]:
#importing modules
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re


from langdetect import detect_langs
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer



In [ ]:
#twitter api keys
auth = tweepy.OAuth1UserHandler(
   )

api = tweepy.API(auth, wait_on_rate_limit=True)


In [ ]:
search_words = "#wakandaforever OR #WakandaForever OR #blackpanther OR #WakandaForeverID OR #BlackPantherWakandaForever OR #BlackPanther OR #BlackPanther2"
search_query = search_words + " -filter:retweets AND -filter:replies"


limit = 100000
tweets = tweepy.Cursor(api.search_tweets, q=search_query, tweet_mode='extended').items(limit)

data=[]
for tweet in tweets:
    data.append([tweet.id,tweet.user.screen_name, tweet.created_at, tweet.full_text,tweet.user.location, tweet.retweet_count,tweet.favorite_count, tweet.user.followers_count, tweet.user.friends_count, tweet.user.verified, tweet.source])

In [ ]:
columns =['id','username','time_of_tweet', 'tweet', 'location', 'retweets', 'likes', 'followers', 'following', 'verified', 'tweet_source']
bp_tweets=pd.DataFrame(data, columns=columns)
bp_tweets.to_csv("BlackPanther-24-11.csv")

In [4]:
tweets = pd.read_csv('BlackPanther-24-11.csv')
tweets.sample(10)

,Unnamed: 0,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source
27702,27702,1593058712780148736,LEJerni13,2022-11-17 01:49:29+00:00,Uhm. Yeah. So I fully ugly cried through appro...,NaN,0,0,727,877,False,Twitter for iPhone
33223,33223,1592748164368699394,EIianaLuna,2022-11-16 05:15:28+00:00,Un wáas k’oop al que diga que Kukulcán es Azte...,NaN,0,1,484,596,False,Twitter for iPhone
65,65,1595736832956067841,abdulvhad,2022-11-24 11:11:22+00:00,Killmonger showing up in #WakandaForeverID was...,NaN,0,1,4608,3230,False,Twitter for Android
14428,14428,1594109767487836160,ms_mimetica,2022-11-19 23:26:00+00:00,Estamos de acuerdo en que Okoye con su estilaz...,NaN,0,0,7,319,False,Twitter for Android
13146,13146,1594186657309560832,notladyblu,2022-11-20 04:31:32+00:00,#WakandaForever My husband wanted to see it. I...,NaN,0,2,1690,2255,False,Twitter for Android
14849,14849,1594088136845688833,Cinepolis,2022-11-19 22:00:03+00:00,Los reales y yo llegando a ver #WakandaForever...,México,17,207,8163877,20721,True,Emplifi
22477,22477,1593431042450407425,toryhargro,2022-11-18 02:28:59+00:00,#Shuri has the most compelling hero’s journey ...,"Atlanta, GA",1,3,2657,2813,True,Twitter for iPhone
32005,32005,1592839814893957122,onntomorina,2022-11-16 11:19:40+00:00,พรุ่งนี้ day off ไปดู #WakandaForever อีกรอบดี...,NaN,0,3,2184,993,False,Twitter for Android
346,346,1595663039562498050,cinemaecritica,2022-11-24 06:18:09+00:00,Amsterdam: Il Culto della Libertà\nScopri l’Ot...,NaN,0,1,226,50,False,Twitter Web App
27988,27988,1593041673017511936,4lexandria__,2022-11-17 00:41:46+00:00,#WakandaForever had me ugly crying.,77(3)12,0,0,287,391,False,Twitter for iPhone


In [ ]:
tweets.info()

In [ ]:
tweets.head()

In [5]:
tweets.drop(columns='Unnamed: 0', inplace = True)

In [99]:
tweets['location'].fillna('None', inplace=True)

In [7]:
tweets['time_of_tweet'] = tweets['time_of_tweet'].astype('datetime64')

In [8]:
def get_hash(tweet):
    tweet = re.findall(r'\#\w+', tweet)
    return " ".join(tweet)

In [9]:
tweets['hashtags'] = tweets['tweet'].apply(get_hash)


In [10]:
hashtags = pd.Series(" ".join(tweets['hashtags']).split()).value_counts()
#hashtags.to_csv('hashtag_freq.csv')
hashtags[:10]

#WakandaForever                20990
#BlackPanther                   9316
#BlackPantherWakandaForever     5887
#wakandaforever                 1997
#Namor                          1746
#blackpanther                   1297
#Marvel                         1211
#BlackPanther2                   998
#WakandaForeverID                978
#MarvelStudios                   808
dtype: int64

In [11]:
def get_languages(tweet):
    try:
        tweet = detect_langs(tweet)
    except: 
        tweet = "nolang"
    tweet = str(tweet).split(':')[0][1:]
    return tweet

In [12]:
tweets['language'] = tweets['tweet'].apply(get_languages)

In [13]:
lang_code = pd.read_csv('language_codes.csv')
dict_lang_code = dict(zip(lang_code['alpha2'], lang_code['English']))
tweets['language'] = tweets['language'].replace(dict_lang_code)

In [14]:
tweets.language.value_counts()[:10]

English       24248
Spanish        3315
Danish         1472
Portuguese     1464
French          768
Indonesian      505
Japanese        470
Thai            470
Norwegian       386
Italian         326
Name: language, dtype: int64

In [15]:
def get_links(tweets):
    link = re.findall(r'((http|ftp|https):\/\/)(([\w.-]*)\.([\w]*))', tweets)
    if len(link) < 1:
        return 'No'
    else: return 'Yes'


In [16]:
tweets['contains_media'] = tweets['tweet'].apply(get_links)

In [17]:
tweets.contains_media.value_counts()

Yes    20872
No     14182
Name: contains_media, dtype: int64

In [18]:
cast = ['namor', 'shuri', 'ironheart', 'ramonda', 'namora', 
        'okoye', 'aneka', 'nakia', 'mbaku', 'attuma', 'everett', ]


def get_cast(tweets):
    tweets = tweets.lower()
    tokens = word_tokenize(tweets)
    cast_names = [name for name in tokens if name in cast]
    cast_names_title = [name.title() for name in cast_names]
    return " ".join(cast_names_title)

In [19]:
tweets['cast_name'] = tweets['tweet'].apply(get_cast)

In [20]:
names = pd.Series(" ".join(tweets['cast_name']).split()).value_counts()
# names.to_csv('names_freq.csv') ##save the generated dataframe
names[:10]

Namor        3953
Shuri        1985
Okoye         348
Namora        295
Ironheart     288
Nakia         220
Ramonda       216
Mbaku         202
Attuma        123
Aneka          53
dtype: int64

In [21]:
tweets['tweet'] = tweets['tweet'].replace('\n', ' ', regex=True)


In [37]:
sid = SentimentIntensityAnalyzer()
tweets['sentient_scores'] = tweets['tweet'].apply(sid.polarity_scores)
tweets['sentient_scores'] = tweets['sentient_scores'].apply(lambda x: x['compound'])

In [48]:
def get_sentiment(tweet):
    if tweet > 0:
        return "Positive"
    elif tweet == 0:
        return np.nan
    else: return "Negative"

In [49]:
tweets['sentiment'] = tweets['sentient_scores'].apply(get_sentiment)

In [50]:
tweets['sentiment'].value_counts()

Positive    12441
Negative     5168
Name: sentiment, dtype: int64

In [113]:
tweets['location'].value_counts()[:50]

None                         10470
Los Angeles, CA                495
United States                  384
Nigeria                        193
London, England                179
México                         176
UK                             171
New York, NY                   168
Atlanta, GA                    164
India                          152
California, USA                144
Brasil                         139
Chicago, IL                    137
United Kingdom                 115
London                          99
España                          98
Canada                          98
Paris, France                   95
 Bonaire Islands 🏙🏖🏝 / DF       93
New York, USA                   88
Houston, TX                     86
São Paulo, Brasil               84
Brooklyn, NY                    81
Florida, USA                    80
USA                             78
France                          77
New York                        74
she/her                         72
Washington, DC      

In [131]:
def get_country(inc_c, case, cor_c):
    tweets.loc[tweets['c_location'].str.contains(inc_c, case=case), 'c_location'] = cor_c


In [138]:
tweets['c_location'] = tweets['location']


dicts = {'brazil':'Brazil', 'brasil':'Brazil', 'nigeria':'Nigeria', 'méxico': 'Mexico', 'france':'France', 'india':'India', 'london':'United Kingdom', 'UK':'United Kingdom', 'england':'United Kingdom','Bonaire':'Bonaire Islands', 'España':'Spain', 'thailand':'Thailand', 'Ohio':'United States', 'South Africa':'South Africa'}
for key, val in dicts.items():
    get_country(key, False, val)


list_us = ['USA', 'KS', 'FL', 'CA', 'GA', 'AZ', 'CO','IL', 'TX', 'PA', 'DC', 'TN', 'NY', 'WA', 'NYC', 'NC', 'NV', 'OH', 'LA', 'MA', 'Los Angeles' 'new york','New York', 'Los Angeles']
for states in list_us:
    get_country(states, True, 'United States')

tweets['c_location'].replace('None', np.nan, inplace=True)
tweets['c_location'].value_counts()[:50]

United States                    5768
United Kingdom                   1123
Brazil                            666
Mexico                            582
India                             572
France                            311
Spain                             308
Nigeria                           193
South Africa                      190
Thailand                          104
Canada                             98
Bonaire Islands                    93
she/her                            72
Talokan                            61
Colombia                           60
Wakanda                            59
Earth                              59
Indonesia                          55
Toronto, Ontario                   54
Inside the Multiverse              52
Location: Classified               51
Madrid                             49
nerdtalks22@gmail.com              47
Argentina                          47
(📩): aassemblebr@outlook.com       46
Nairobi, Kenya                     45
Malaysia    

In [137]:
tweets.sample(20)

,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source,hashtags,language,contains_media,cast_name,sentient_scores,sentiment,c_location
15829,1594029490874744834,sfstandard,2022-11-19 18:07:00,E-40 never stops hustling. In addition to his...,"San Francisco, CA",0,2,10698,497,True,Sprout Social,#WakandaForever,English,Yes,,0.0114,Positive,United States
32251,1592816522371936257,ab0a2e8604b549f,2022-11-16 09:47:06,Bastet was really at a kick back flexing with ...,Oakland California,13,65,593,2021,False,Twitter for Android,#BlackPanther #namor #Shuri #RecastTchalla,English,Yes,Namor Shuri,0.6705,Positive,Oakland California
4432,1595081256282341377,VibinWithViolet,2022-11-22 15:46:21,Celebrated our 5 year anniversary yesterday wi...,"Houston,Texas",1,14,9420,750,False,Twitter Web App,#BlackPanther,English,No,,0.5719,Positive,"Houston,Texas"
31447,1592878763846819843,DoshQuincy,2022-11-16 13:54:26,Remember on #BreakingBad when Jane's dad went ...,None,0,1,258,87,False,Twitter Web App,#BreakingBad #Wakandaforever,English,No,,-0.6597,Negative,NaN
7513,1594686947573829633,says_twit,2022-11-21 13:39:30,I've only seen #WakandaForever once so far. It...,None,0,0,2,56,False,Twitter Web App,#WakandaForever #reversedbechdeltest #bechdeltest,English,No,,-0.4215,Negative,NaN
13552,1594162466291732480,nightsir3000,2022-11-20 02:55:24,I just watched Black Panther: Wakanda Forever ...,Brasil,0,0,140,303,False,Trakt.tv,#BlackPantherWakandaForever #trakt,English,Yes,,0.0,NaN,Brazil
13062,1594194032221368322,DMason53,2022-11-20 05:00:50,Black Panther 2 is all-time! That first 45 min...,None,0,1,120,189,False,Twitter for iPhone,#BlackPanther,English,No,,0.8217,Positive,NaN
8889,1594510490574770179,FIsaacbonilla,2022-11-21 01:58:20,Wakanda forever 🙌🙌 #WakandaForever https://t.c...,Panamá. Panamá,0,0,120,203,False,Twitter for Android,#WakandaForever,Danish,Yes,,0.0,NaN,Panamá. Panamá
29119,1592988887307390981,MrJohnGarlow,2022-11-16 21:12:01,Change of plans for Friday I'll be at Meadowto...,"Maple Ridge, British Columbia",0,2,579,2455,False,Twitter for Android,#BlackPantherWakandaForever,English,No,,0.2714,Positive,"Maple Ridge, British Columbia"
29137,1592988304072667136,The_one_happy,2022-11-16 21:09:42,NUNCA MAISSS SÃO 21:30 PARA VER FINALMENTE O M...,"Lisboa, Portugal",0,1,1927,937,False,Twitter for iPhone,#BlackPanther #WakandaForever,English,No,,0.0,NaN,"Lisboa, Portugal"


In [ ]:
cv = tweets['location'].replace(np.nan, "None")

In [ ]:
cv[cv.str.contains('IL')][:60]
